<a href="https://colab.research.google.com/github/matech96/DeepLearningAVizualisInformatikaban/blob/master/HF1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning
!pip install --upgrade tqdm==4.41.1

     |████████████████████████████████| 6.0MB 9.1MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 829kB 77.7MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.7.1-cp36-none-any.whl size=145306 sha256=50d6d34a79d7920e4e5056e1f92960880fd9e4daa893263928536e9863716248
  Stored in directory: /root/.cache/pip/wheels/dc/93/61/14094d2116ff739513dda993007501ae5701b78386b39d5912
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=8ba2dc595beff03c82dd2d35d1c68d4446d0171aaeb86d3a53e8bf1d9c716b5f
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built pytorch-lightning future
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


     |████████████████████████████████| 61kB 4.9MB/s 
  Found existing installation: tqdm 4.43.0
    Uninstalling tqdm-4.43.0:
      Successfully uninstalled tqdm-4.43.0


#Deep Learning Alkalmazása a Vizuális Informatikában 
##1. Házi feladat

**Fontos tudnivalók**

*   A házi feladatot önállóan kell elvégezni. Teljes megoldások máshonnan történő másolása nem elfogadható.
*   Ez a notebook egyben megoldás és dokumentáció. Az eredményeket értékeljétek ki, printelgessetek, ahhoz fűzzetek szépen megszerkesztett kommentárokat, stb.
*   Egy rész feladatot nem kell egyetlen kód blokkban megoldani. Sokkal szebb és struktúráltab lesz a munka, ha logikailag összetartozó egységekre bontjátok és azokat szépen egyesével magyarázzátok.
*   A házi feladat elkészítésére 3 hét áll rendelkezésetekre. Beadni az Edu portálon lehet. A határidő Március 12. 12:00.

### 1. Rész

Implementálj PyTorch-ban 2D konvolúciós réteget az NN, valamint a Functional modulok használata nélkül. A réteg konstruktorának a következő bemeneti paraméterei legyenek:


*   Kernel méret (n)
*   Bemeneti csatornaszám (inCH)
*   Kimeneti csatornaszám (oCH)

A rétegnek legyen forward függvénye, amelyben megkap egy [inCH x H x W] méretű tenzort, ahol H és W nem ismertek előre. A kimenete pedig legyen [oCH x H x W] (használj a kernel mérethez megfelelő paddinget).



In [0]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt


class Conv():
  def __init__(self, n, inCH, oCH):
    self.n = n
    self.inCH = inCH
    self.oCH = oCH
    self.W = torch.Tensor(np.random.randn(oCH,inCH,n,n)).requires_grad_(True)

  def parameters(self):
    yield self.W

  def forward(self, x):
    pad = self.n // 2
    x_ = torch.zeros(1,self.inCH,x.shape[2]+2*pad,x.shape[3]+2*pad).requires_grad_(False)
    if pad != 0:
      x_[0, :, pad:-pad, pad:-pad] = x
    else:
      x_ = x
    res = torch.zeros(1,self.oCH,x.shape[2],x.shape[3]).requires_grad_(False)
    for i in range(x_.shape[2] - self.n + 1):
      for j in range(x_.shape[3] - self.n + 1):
        res[0, :, i, j] = torch.addmv(torch.zeros(self.oCH), torch.reshape(self.W, (self.oCH, -1)), torch.reshape(x_[0, :, i:i+self.n, j:j+self.n], ((-1, ))))
    return res

  

Megnézem, hogy mindenova kerül-e érték.

In [0]:
myconv = Conv(n = 5, inCH = 3, oCH = 1)

x = torch.Tensor(np.random.randn(1,3,8,8)).requires_grad_(False)
plt.imshow(myconv.forward(x).detach().numpy()[0,0,:,:])

###2. Rész

Amennyiben az implementáció elkészült, ellenőrizd az implementáció helyességét a PyTorch Conv2d modulja segítségével! (Tipp: Használj többfajta konfigurációt!)
https://pytorch.org/docs/stable/nn.html#conv2d

Egyenlővé teszem a kerneleket és vizuálisan összehasonlítom az eredményt.

In [0]:
n = 5
inCH = 3
oCH = 1
W = torch.Tensor(np.random.randn(oCH,inCH,n,n))
x = torch.Tensor(np.random.randn(1,3,8,8)).requires_grad_(False)

myconv = Conv(n = 5, inCH = 3, oCH = 1)
with torch.no_grad():
  myconv.W = nn.Parameter(W)
plt.subplot(1,2,1)
plt.imshow(myconv.forward(x).detach().numpy()[0,0,:,:])

refconv = nn.Conv2d(3, 1, (5, 5), padding=(2, 2), bias=False)
with torch.no_grad():
  refconv.weight = nn.Parameter(W)
plt.subplot(1,2,2)
plt.imshow(refconv(x).detach().numpy()[0,0,:,:])
plt.show()

Numerikus összehasonlítást végzek néhány jellemző konfigurációra.

In [0]:
for n in [1,3,5,7]:
  for inCH in [1,3,8,16]:
    for oCH in [1,3,8,16]:
      W = torch.Tensor(np.random.randn(oCH,inCH,n,n))
      x = torch.Tensor(np.random.randn(1,inCH,8,8)).requires_grad_(False)

      myconv = Conv(n, inCH, oCH)
      with torch.no_grad():
        myconv.W = nn.Parameter(W)
      myres = myconv.forward(x).detach().numpy()

      refconv = nn.Conv2d(inCH, oCH, (n, n), padding=(n//2, n//2), bias=False)
      with torch.no_grad():
        refconv.weight = nn.Parameter(W)
      refres = refconv(x).detach().numpy()

      assert(np.all((myres - refres) < 1e-4))

###3. Rész

Hozz létre egy egy bemeneti csatornás konvolúciós réteget, amelynek az egyes kimeneti csatornákhoz tartozó szűrőinek súlyait inicializáld véletlenszerűen. (A kimeneti csatornák száma legyen mondjuk 3.)

Ezt követően olvass be egy képet, szürkeárnyalatosítsd, és futtass le OpenCV segítségével 3 általad választott konvolúciós szűrőt!

Végezetül tanítsd be arra a létrehozott konvolúciós réteget, hogy az OpenCV-s szűréseket minél pontosabban reprodukálni tudja. Vizsgáld meg az eredményként kapott szűrőket! (Tipp: Az OpenCV-vel szűrt képeket használd referenciaként, mondjuk a négyzetes hiba költséggel. Több képet egy tömbbe összepakolni np.stack segítségével lehet.)

In [0]:
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset
from torchvision.datasets import CIFAR10
from torchvision import transforms
import os
from PIL import ImageFilter

In [0]:
class GaussCifar(Dataset):
  def __init__(self, offset=0):
    self.cifar = CIFAR10(os.getcwd() + '/cifar',download=True)
    self.offset = offset

  def __getitem__(self, i):
    index = i + self.offset
    chFirst = lambda x: np.rollaxis(np.array(x, dtype=np.float32) / 255., 2, 0)
    return chFirst(self.cifar[index][0]), chFirst(self.cifar[index][0].filter(ImageFilter.Kernel((3, 3), 
          (-1, -1, -1, -1, 9, -1, -1, -1, -1), 1, 0)))
    
  def __len__(self):
    return 64

In [0]:
dl = DataLoader(GaussCifar(), batch_size=64)

In [0]:
len(dl)

In [0]:
t = next(iter(dl))
plt.imshow(np.rollaxis(t[0][0, ].detach().numpy(), 0, 3))
plt.show()
plt.imshow(np.rollaxis(t[1][0, ].detach().numpy(), 0, 3))
plt.show()

In [0]:
import pytorch_lightning as pl
from torch.optim import Adam
class CoolCIFAR(pl.LightningModule):  
  def __init__(self, model):
    super(CoolCIFAR, self).__init__()
    self.model = model
    self.mse = nn.MSELoss()

  def parameters(self):
    return self.model.parameters()

  def forward(self, x):
    return self.model(x)

  def train_dataloader(self):
    return DataLoader(GaussCifar(), batch_size=64)

  # def val_dataloader(self):
  #   return DataLoader(GaussCifar(64), batch_size=64)

  def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-3)

  def training_step(self, batch, batch_idx):
    x, y = batch
    res = self.forward(x)
    loss = self.mse(res, y)
    logs = {'train_loss': loss}
    return {'loss': loss, 'log': logs}

  # def validation_step(self, batch, batch_idx):
  #   x, y = batch
  #   res = self.forward(x)
  #   loss = self.mse(res, y)
  #   return {'val_loss': loss}

  # def validation_end(self, outputs):
  #   avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
  #   tensorboard_logs = {'val_loss': avg_loss}
  #   return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

In [0]:
from pytorch_lightning import Trainer

model = CoolCIFAR(nn.Conv2d(3, 3, (3, 3), padding=(3//2, 3//2), bias=False))
trainer = Trainer(max_epoch=1)
trainer.fit(model)